## Data Source:

https://data.open-power-system-data.org/renewable_power_plants/2020-08-25

## Step1. Data Cleaning

In [ ]:
import pandas as pd

# Define the file path and column data types
file_path = '/renewable_power_plants_EU.csv'
dtypes = {'energy_source_level_3': str, 'commissioning_date': str}

# Define the chunk size for reading in the data
chunk_size = 100000

# Initialize an empty list to store the filtered data
filtered_data = []

# Iterate through the data in chunks
for chunk in pd.read_csv(file_path, dtype=dtypes, chunksize=chunk_size):
    
    # Filter the columns to keep
    keep_cols = ['electrical_capacity', 'energy_source_level_2', 'technology', 
                 'nuts_1_region', 'nuts_2_region', 'nuts_3_region', 'lon', 'lat',
                 'municipality', 'country', 'commissioning_date']
    chunk = chunk[keep_cols]

    # Filter the rows
    chunk = chunk[chunk['energy_source_level_2'] == 'Wind'] # Only keep rows where energy_source_level_2 is 'Wind'
    
    # Drop duplicates
    chunk = chunk.drop_duplicates()

    # Append the filtered data to the list
    filtered_data.append(chunk)

# Concatenate the filtered data into a single DataFrame
df = pd.concat(filtered_data, ignore_index=True)

# Save cleaned data to a new CSV file
df.to_csv('/wind_energy_EU_cleaned.csv', index=False)

## Step2. Data Storage

In [ ]:
import pandas as pd
import psycopg2

# Connect to the PostgreSQL database
conn = psycopg2.connect(database="mydatabase", user="myuser", password="mypassword", host="localhost", port="5432")

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv('/wind_energy_EU_cleaned.csv')

# Insert the data into a PostgreSQL table
df.to_sql('mytable', conn, if_exists='replace')